<a href="https://colab.research.google.com/github/zhensongren/carto_bigquery_public_census_data_analysis/blob/master/Geospatial_data_analysis_of_Census_ACS_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Median household income in Greater Houston area
Greater Houston, designated as Houston–The Woodlands–Sugar Land, is the 5th-most populous metropolitan statistical area in the United States.[Greater Houston-wikipedia](https://en.wikipedia.org/wiki/Greater_Houston)

Here, we're looking at median income in the most recent year(2018) in historical ACS data (2007 - 2018) provided by CARTO and made publicly available as part of [BigQuery's Public Datasets](https://cloud.google.com/bigquery/public-data/).

For visualization we're joining with [CARTO's custom shoreline-clipped version](https://carto.com/blog/tiger-shoreline-clip/) of the U.S. Census' TIGER boundaries. These are also publicly accessible as part of CARTO's own public dataset initiative in the `carto-do-public-data` BQ project.

In [0]:
!pip install cartoframes==1.0b3
!pip install geopandas

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

## Get the data from BigQuery

In [0]:
import geopandas as gpd

from cartoframes.viz.helpers import color_bins_layer
from google.cloud import bigquery
from shapely import wkt

####### REPLACE THIS WITH YOUR OWN GCLOUD PROJECT YOU WANT TO BE BILLED####
bq_client = bigquery.Client('carto-do')

q = '''
WITH acs_2017 AS (
  SELECT geo_id, median_income AS median_income_2017
  FROM `bigquery-public-data.census_bureau_acs.blockgroup_2017_5yr`  
  WHERE geo_id LIKE '36047%'
),

acs_2010 AS (
  SELECT geo_id, median_income AS median_income_2010
  FROM `bigquery-public-data.census_bureau_acs.blockgroup_2010_5yr` 
  WHERE geo_id LIKE '36047%'
),

acs_diff AS (
  SELECT
    a17.geo_id, a17.median_income_2017, a10.median_income_2010, geo.geom,
    a17.median_income_2017 - a10.median_income_2010 AS median_income_diff
  FROM acs_2017 a17
  JOIN acs_2010 a10
    ON a17.geo_id = a10.geo_id
  JOIN `carto-do-public-data.usa_carto.geography_usa_blockgroupclipped_2015` geo
    ON a17.geo_id = geo.geoid
)

SELECT * FROM acs_diff WHERE median_income_diff IS NOT NULL
'''

df = bq_client.query(q).to_dataframe()
gdf = gpd.GeoDataFrame(df, geometry=df.geom.apply(wkt.loads))

color_bins_layer(gdf, value='median_income_diff',palette='[#64B97A,#DFF873,#f54260]',bins=10,legend=False, widget=True)

In [0]:
# Much more complex query calculating 
# polyfilling with H3 hexagon cells, 
# Calculating the value/num of cells within the polygon
# making a Kring to smooth borders
q = '''
with data as (
select 
  g.geoid,
  jslibs.h3.ST_H3_POLYFILLFROMGEOG(g.geom,10) as h3ids,
  ARRAY_LENGTH(jslibs.h3.ST_H3_POLYFILLFROMGEOG(g.geom,10)) as num_h3,
  total_pop
  from `carto-do-public-data.usa_carto.geography_usa_blockgroupclipped_2015` as g
  inner join  `bigquery-public-data.census_bureau_acs.blockgroup_2017_5yr` as s 
  on g.geoid=s.geo_id
  WHERE g.geoid LIKE '36047%'
),

--Assign to each grid the amount proportionally
h3assigned AS (
SELECT h3,total_pop/num_h3 as total_pop_n,total_pop,num_h3
FROM data,UNNEST(h3ids) as h3
),

h3kgrids AS(
SELECT jslibs.h3.kRing(h3,2) as kgrids,total_pop_n,total_pop,num_h3 
FROM h3assigned),

un_kgrid AS(
SELECT grids,total_pop_n,total_pop,num_h3  FROM h3kgrids,UNNEST(kgrids) as grids),

layer AS (
SELECT jslibs.h3.ST_H3_BOUNDARY(grids) as geom,
ROUND(avg(total_pop_n),3) as total_pop_n,
avg(total_pop) as orig_total_pop
FROM un_kgrid group by grids)

SELECT * FROM layer

'''


df = bq_client.query(q).to_dataframe()
gdf = gpd.GeoDataFrame(df, geometry=df.geom.apply(wkt.loads))

In [0]:
#visualize it on CartoFrames
color_bins_layer(gdf, value='total_pop_n',palette='[#ffffff,#64B97A,#DFF873,#f54260,#e20aff]',bins=10,stroke_color="transparent",legend=False, widget=True)